In [2]:
import nbimporter
import numpy as np
import random
import time
import math

# import chromosome as Chr

In [3]:
from chromosome import Chromosome

In [4]:
base = Chromosome(np.arange(3))
base.get_positions()

array([0, 1, 2])

In [5]:
class EvolManager:
    def __init__(
        self,
        genes_per_chrom,
        pop,
        generations,
        reproductive_pool_size = None,
        offspring = None,
        mutation_rate = 0.2,
        reproductive_coefficient = 1 / 3,
        ):

        if reproductive_pool_size is None:
            reproductive_pool_size = math.floor(pop * 0.8)

        if offspring is None:
            offspring = math.floor(reproductive_pool_size * 1.5)

        self.__n = genes_per_chrom
        self.__population = pop
        self.__generations = generations
        self.__current_generation = 0
        self.__chrom_array = []
        self.__solutions = []
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool = []
        self.__sorted_chrom_array = []
        self.__reproductive_pool_size = reproductive_pool_size
        self.__offspring = []
        self.__offspring_size = int(offspring)
        self.__big_array = []
        self.__solution_times = np.array([])
        self.__mutation_rate = mutation_rate
        self.__reproductive_coefficient = reproductive_coefficient
        self.__solution_times = np.array([], dtype = float)

        self.__starting_time = None

        self.populate()


    def populate(self):
        proto_chrom = Chromosome(list(range(self.__n)))

        for i in range(self.__population):
            self.__chrom_array.append( 
                Chromosome(proto_chrom.random_vec()) )

        for i in range(len(self.__chrom_array)):
            new_sol = True
            if self.__chrom_array[i].fitness() == 1.00:
                for j in range(len(self.__solutions)):
                    if np.array_equal( self.__chrom_array[i].get_positions(),
                        self.__solutions[j].get_positions() ):

                        new_sol = False
                        break
                if new_sol:
                    self.__solutions.append(self.__chrom_array[i])
                    self.__sol_generations = np.append(self.__sol_generations, 
                        self.__current_generation)
                    self.__solution_times = np.append(self.__solution_times, 0)
        print("It's populated over here!")


    def get_chrom_array(self):
        return self.__chrom_array


    def get_solutions(self):
        return self.__solutions


    def get_solution_times(self):
        return self.__solution_times

    
    def get_sol_generations(self):
        return self.__sol_generations


    def get_best_chrom_fitness(self):
        ordered_popul = sorted( self.__chrom_array, 
            key = lambda chrom: - chrom.fitness() )
        return ordered_popul[0].fitness()


    def get_best_chrom_mutual_threats(self):
        ordered_popul = sorted( self.__chrom_array, 
            key = lambda chrom: - chrom.fitness())
        return ordered_popul[0].mutual_threats()


    def get_reproductive_pool(self):
        return self.__reproductive_pool


    def get_offspring(self):
        return self.__offspring

    
    def get_current_generation(self):
        return self.__current_generation


    def set_chrom_array(self, vec_array):
       
        self.__n = len(vec_array[0])
        self.__population = len(vec_array)
        self.__current_generation = 0
        self.__chrom_array = []
        self.__solutions = []
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool = []
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([])

        for i in vec_array:
            self.__chrom_array.append( Chromosome(i) )


    # show_chromosomes
    def show_chromosomes_positions(self):
        print("__chrom_array members: ")
        for i in self.__chrom_array:
            print(f"{i.get_positions()}", end = " ") 
            print(f"fitness: {i.fitness()}")



    def show_solutions(self):
        print("Solutions: ")
        for i in range(len(self.__solutions)):
            print(f"solution [{i}]: {self.__solutions[i].get_positions()}. \
                    Generation: {self.__sol_generations[i]}")

        if len(self.__solutions) > 2:
            if ( self.__solutions[len(self.__solutions) - 1].get_positions() 
                    == self.__solutions[len(self.__solutions) 
                    - 2].get_positions() ).all():
                print("last solution repeated")


    def make_reproductive_pool(self):
        self.__reproductive_pool.clear()

        # Sort chromosomes by fitness in descending order
        self.__sorted_chrom_array = sorted(self.__chrom_array, 
                key = lambda crom: - crom.fitness())

        c = 0
        while len(self.__reproductive_pool) < self.__reproductive_pool_size:
            bool_signal = False
            d = c % len(self.__sorted_chrom_array)

            if ( len(self.__sorted_chrom_array ) <= (self.__population / 2) ):
                self.__sorted_chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )

            elif self.__n <= 2 or ( c > self.__population and len(self.__reproductive_pool) == 0 ):
                self.__reproductive_pool = self.__sorted_chrom_array[:self.__reproductive_pool_size]

            # Chromosomes have a probability of entering the reproductive pool proportional to their fitness,
            # unless a entire iteration was made without adding any chromosome to the reproductive pool.
            # Chromosomes with fitness = 0 usually don't enter the reproductive pool.
            elif random.random() < (self.__sorted_chrom_array[ d ].fitness() * self.__reproductive_coefficient):
                self.__reproductive_pool.append(self.__sorted_chrom_array[ d ])

            # If a solution is found, it is removed from the reproductive pool 
            # and a random chromosome is added to the population.
            if self.__sorted_chrom_array[ d ].fitness() == 1.000000:
                if len(self.__solutions) == 0:
                    self.__solutions.append(self.__sorted_chrom_array[ d ])
                    self.__sol_generations = np.append(self.__sol_generations, self.__current_generation)
                    ending = time.perf_counter()
                    lapse = ending - self.__starting_time
                    self.__solution_times= np.append(self.__solution_times, lapse)
                    print("Time for first solution:")
                    print(lapse)

                    # self.__reproductive_pool.append(self.__sorted_chrom_array[ d ].make_child())

                    self.__chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )
                    self.__chrom_array.remove(self.__sorted_chrom_array[ d ])
                    self.__sorted_chrom_array.pop( d )
                    
                    continue
                
                
                for i in range( len(self.__solutions) ):
                    if ( self.__solutions[i].get_positions() 
                                == self.__sorted_chrom_array[ d ].get_positions() ).all():
                        self.__chrom_array.append( Chromosome( self.__chrom_array[0].random_vec() ) )
                        # self.__reproductive_pool.append(self.__sorted_chrom_array[ d ].make_child())
                        if ( self.__sorted_chrom_array[ d ] in self.__chrom_array ):
                            self.__chrom_array.remove(self.__sorted_chrom_array[ d ])
                        
                        

                        if ( self.__sorted_chrom_array[ d ] in self.__reproductive_pool ):
                            self.__reproductive_pool.remove(self.__sorted_chrom_array[ d ])

                        self.__sorted_chrom_array.pop( d )

                        bool_signal = True

                        break
                
                if bool_signal:
                    continue

                else:
                    self.__solutions.append(self.__sorted_chrom_array[ d ])
                    self.__sol_generations = np.append(self.__sol_generations, self.__current_generation)
                    ending = time.perf_counter()
                    lapse = ending - self.__starting_time
                    self.__solution_times = np.append(self.__solution_times, lapse)
                    print(f"Time for new solution: { lapse }")

            c += 1



    def reproductive_season(self):
        self.__offspring.clear()
        print(f"self.__offspring_size: {self.__offspring_size}")
        for i in range (self.__offspring_size):
            self.__offspring.append( self.__reproductive_pool
                [i % len(self.__reproductive_pool) ].make_child() )

    
    def replacement(self):
        self.__big_array = self.__chrom_array + self.__offspring
        self.__big_array = sorted(self.__big_array, key = lambda crom: - crom.fitness())
        self.__chrom_array = self.__big_array[:self.__population]


    def greedy_next_generation(self):
        print(f"Current generation: {self.__current_generation}")
        
        self.make_reproductive_pool()
        self.reproductive_season()
        self.replacement()


    def greedy_evolution(self):
        self.__current_generation = 0
        self.__chrom_array.clear()
        self.__solutions.clear()
        self.__sol_generations = np.array([], dtype = int)
        self.__reproductive_pool.clear()
        self.__sorted_chrom_array.clear()
        self.__offspring.clear()
        self.__big_array.clear()
        self.__solution_times = np.array([], dtype = float)

        self.populate()
        self.__starting_time = time.perf_counter()

        while self.__current_generation < self.__generations:
            self.greedy_next_generation()
            self.__current_generation += 1





        print(f"Current generation: {self.__current_generation}")
        print(f"Population: {self.__population}")
        self.show_chromosomes_positions()
        self.show_solutions()



  
        




# Testing

def __init__(  
> self,  
        genes_per_chrom,  
        pop,  
        generations,  
        reproductive_pool_size = None,  
        offspring = None,  
        mutation_rate = 0.2,  
        reproductive_coefficient = 1 / 3,
        ):

## _ _ init _ _ ( ), populate(), get_chrom_array()

In [6]:
evol_manager = EvolManager(5, 5, 10)
for c in evol_manager.get_chrom_array():
    print(c.get_positions())

SOLUTION FOUND: [1 4 2 0 3] It's populated over here!
[0 1 3 2 4]
[0 1 3 2 4]
[1 4 2 0 3]
[2 3 0 4 1]
[2 3 1 4 0]


## get_solutions()

In [7]:
evol_manager = EvolManager(5, 50, 10)
for c in evol_manager.get_chrom_array():
    print(c.get_positions())

SOLUTION FOUND: [2 4 1 3 0] SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [1 4 2 0 3] It's populated over here!
[0 2 3 1 4]
[2 1 0 4 3]
[0 4 2 3 1]
[2 4 0 1 3]
[1 3 0 4 2]
[2 4 1 3 0]
[1 3 2 0 4]
[2 1 4 0 3]
[1 4 0 2 3]
[0 4 3 2 1]
[0 2 1 4 3]
[2 3 1 4 0]
[2 0 1 3 4]
[2 1 3 0 4]
[1 0 4 3 2]
[0 2 3 4 1]
[1 2 3 4 0]
[0 2 1 3 4]
[0 2 3 4 1]
[1 0 2 3 4]
[0 2 1 4 3]
[1 2 4 3 0]
[1 0 2 3 4]
[0 2 4 3 1]
[0 3 2 4 1]
[0 1 4 3 2]
[1 3 4 0 2]
[0 1 2 4 3]
[0 2 4 1 3]
[0 2 1 3 4]
[1 2 0 3 4]
[2 0 3 1 4]
[1 2 3 4 0]
[1 4 0 3 2]
[1 4 2 3 0]
[0 2 3 1 4]
[2 4 0 1 3]
[0 3 4 1 2]
[0 3 4 2 1]
[0 2 1 4 3]
[1 3 0 4 2]
[0 1 3 2 4]
[2 1 4 3 0]
[0 4 2 3 1]
[1 3 0 4 2]
[0 2 3 4 1]
[2 1 4 0 3]
[1 4 2 0 3]
[1 3 4 2 0]
[2 1 0 4 3]


In [8]:
for c in evol_manager.get_solutions():
    print(c.get_positions())

[2 4 1 3 0]
[0 2 4 1 3]
[2 0 3 1 4]
[1 4 2 0 3]


In [9]:
s0 = Chromosome([2, 0, 3, 1, 4])
s1 = Chromosome([0, 2, 4, 1, 3])
s2 = Chromosome([1, 3, 0, 2, 4])
s3 = Chromosome([1, 4, 2, 0, 3])
s4 = Chromosome([2, 4, 1, 3, 0])
s5 = Chromosome([0, 3, 1, 4, 2])

In [10]:
s0.print_board()

[[ 0 15  0  0  0]
 [ 0  0  0 15  0]
 [15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]]


In [11]:
s1.print_board()

[[15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]
 [ 0  0  0  0 15]
 [ 0  0 15  0  0]]


In [12]:
s2.print_board()

[[ 0  0 15  0  0]
 [15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]
 [ 0  0  0  0 15]]


In [13]:
s3.print_board()

[[ 0  0  0 15  0]
 [15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]
 [ 0 15  0  0  0]]


In [14]:
s4.print_board()

[[ 0  0  0  0 15]
 [ 0  0 15  0  0]
 [15  0  0  0  0]
 [ 0  0  0 15  0]
 [ 0 15  0  0  0]]


In [15]:
s5.print_board()

[[15  0  0  0  0]
 [ 0  0 15  0  0]
 [ 0  0  0  0 15]
 [ 0 15  0  0  0]
 [ 0  0  0 15  0]]


## get_solution_times()

In [16]:
evol_manager = EvolManager(5, 50, 10)
# for c in evol_manager.get_chrom_array():
#     print(c.get_positions())

SOLUTION FOUND: [0 3 1 4 2] SOLUTION FOUND: [2 4 1 3 0] SOLUTION FOUND: [0 2 4 1 3] SOLUTION FOUND: [1 4 2 0 3] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [1 4 2 0 3] SOLUTION FOUND: [1 4 2 0 3] It's populated over here!


In [17]:
evol_manager.get_solution_times()

array([0., 0., 0., 0., 0.])

In [18]:
evol_manager.get_sol_generations()

array([0, 0, 0, 0, 0])

## get_best_chrom_fitness(),  
## get_best_chrom_mutual_threats()

In [19]:
evol_manager = EvolManager(5, 50, 10)
# for c in evol_manager.get_chrom_array():
#     print(c.get_positions())

SOLUTION FOUND: [1 3 0 2 4] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] It's populated over here!


In [20]:
evol_manager.get_best_chrom_fitness()

SOLUTION FOUND: [1 3 0 2 4] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [1 3 0 2 4] 

1.0

In [21]:
evol_manager.get_best_chrom_mutual_threats()

SOLUTION FOUND: [1 3 0 2 4] SOLUTION FOUND: [2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] 

0

## get_reproductive_pool()

## get_offspring()

## set_chrom_array()

In [22]:
evol_manager = EvolManager(5, 5, 10)

evol_manager.show_chromosomes_positions()


It's populated over here!
__chrom_array members: 
[2 4 3 1 0] fitness: 0.8
[2 0 1 3 4] fitness: 0.8
[1 0 4 2 3] fitness: 0.6
[0 3 4 2 1] fitness: 0.8
[1 3 4 0 2] fitness: 0.8


In [23]:
vectors_array = np.array([
    np.array([2, 0, 3, 1, 4]),
    np.array([0, 2, 4, 1, 3]),
    np.array([1, 3, 0, 2, 4]),
    np.array([1, 4, 2, 0, 3]),
    np.array([2, 4, 1, 3, 0]),
    ])

evol_manager.set_chrom_array(vectors_array)
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[2 0 3 1 4] SOLUTION FOUND: [2 0 3 1 4] fitness: 1.0
[0 2 4 1 3] SOLUTION FOUND: [0 2 4 1 3] fitness: 1.0
[1 3 0 2 4] SOLUTION FOUND: [1 3 0 2 4] fitness: 1.0
[1 4 2 0 3] SOLUTION FOUND: [1 4 2 0 3] fitness: 1.0
[2 4 1 3 0] SOLUTION FOUND: [2 4 1 3 0] fitness: 1.0


## show_chromosomes_positions()

In [24]:
evol_manager = EvolManager(5, 5, 10)

It's populated over here!


In [25]:
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[2 4 3 1 0] fitness: 0.8
[0 4 2 3 1] fitness: 0.6
[0 4 1 2 3] fitness: 0.6
[1 3 2 0 4] fitness: 0.8
[0 2 4 3 1] fitness: 0.8


## show_solutions()

In [26]:
evol_manager = EvolManager(5, 50, 10)

SOLUTION FOUND: [1 3 0 2 4] SOLUTION FOUND: [1 4 2 0 3] SOLUTION FOUND: [1 3 0 2 4] It's populated over here!


In [27]:
evol_manager.show_solutions()

Solutions: 
solution [0]: [1 3 0 2 4].                     Generation: 0
solution [1]: [1 4 2 0 3].                     Generation: 0


In [28]:
evol_manager.show_chromosomes_positions()

__chrom_array members: 
[1 0 4 3 2] fitness: 0.6
[0 4 2 3 1] fitness: 0.6
[2 4 3 0 1] fitness: 0.6
[2 0 3 4 1] fitness: 0.8
[1 4 3 2 0] fitness: 0.6
[0 3 1 2 4] fitness: 0.8
[1 3 0 2 4] SOLUTION FOUND: [1 3 0 2 4] fitness: 1.0
[0 2 4 3 1] fitness: 0.8
[0 4 1 2 3] fitness: 0.6
[2 1 0 3 4] fitness: 0.4
[1 4 2 0 3] SOLUTION FOUND: [1 4 2 0 3] fitness: 1.0
[2 3 0 4 1] fitness: 0.8
[1 4 3 2 0] fitness: 0.6
[1 3 2 4 0] fitness: 0.6
[2 0 4 3 1] fitness: 0.8
[1 4 2 3 0] fitness: 0.8
[0 1 3 2 4] fitness: 0.6
[2 3 1 4 0] fitness: 0.8
[1 0 4 3 2] fitness: 0.6
[2 3 1 4 0] fitness: 0.8
[1 0 4 3 2] fitness: 0.6
[0 2 3 4 1] fitness: 0.6
[0 2 1 4 3] fitness: 0.6
[0 4 3 2 1] fitness: 0.4
[0 2 4 3 1] fitness: 0.8
[1 0 2 3 4] fitness: 0.6
[1 2 3 4 0] fitness: 0.4
[0 4 1 3 2] fitness: 0.8
[1 2 3 0 4] fitness: 0.6
[1 0 3 2 4] fitness: 0.6
[1 4 0 3 2] fitness: 0.8
[2 1 4 0 3] fitness: 0.8
[1 2 0 3 4] fitness: 0.8
[0 4 1 2 3] fitness: 0.6
[1 3 2 0 4] fitness: 0.8
[2 3 4 1 0] fitness: 0.4
[2 0 1 4 3] fitness:

## make_reproductive_pool() & sorting by fitness

In [29]:
chrom_list = []
chr = Chromosome( np.arange(8) )

for i in range(5):
    chrom_list.append( Chromosome( chr.random_vec() ) )

for c in chrom_list:
    print(f"{c.get_positions()} fitness: {c.fitness()}")

[0 6 3 5 2 4 7 1] fitness: 0.9285714285714286
[3 6 0 2 5 4 7 1] fitness: 0.8928571428571429
[2 3 6 7 4 1 0 5] fitness: 0.8571428571428571
[3 5 4 7 6 1 0 2] fitness: 0.6785714285714286
[0 3 4 5 6 7 2 1] fitness: 0.5357142857142857


In [30]:
sorted_chrom_list = sorted(chrom_list, key=lambda crom: -crom.fitness())

for c in sorted_chrom_list:
    print(f"{c.get_positions()} fitness: {c.fitness()}")

[0 6 3 5 2 4 7 1] fitness: 0.9285714285714286
[3 6 0 2 5 4 7 1] fitness: 0.8928571428571429
[2 3 6 7 4 1 0 5] fitness: 0.8571428571428571
[3 5 4 7 6 1 0 2] fitness: 0.6785714285714286
[0 3 4 5 6 7 2 1] fitness: 0.5357142857142857


In [31]:
for i in chrom_list:
    print(i.get_positions())

[0 6 3 5 2 4 7 1]
[3 6 0 2 5 4 7 1]
[2 3 6 7 4 1 0 5]
[3 5 4 7 6 1 0 2]
[0 3 4 5 6 7 2 1]


# Executions

def __init__(  
> self,  
        genes_per_chrom,  
        pop,  
        generations,  
        reproductive_pool_size = None,  
        offspring = None,  
        mutation_rate = 0.2,  
        reproductive_coefficient = 1 / 3,
        ):

## ex 1:


In [32]:
evol_manager = EvolManager(
    genes_per_chrom = 8, 
    pop = 100, 
    generations = 100, 
    reproductive_pool_size = None,
    offspring = 70)

evol_manager.greedy_evolution()

It's populated over here!
SOLUTION FOUND: [2 5 7 0 4 6 1 3] It's populated over here!
Current generation: 0
SOLUTION FOUND: [2 5 7 0 4 6 1 3] SOLUTION FOUND: [2 5 7 0 4 6 1 3] SOLUTION FOUND: [2 5 7 0 4 6 1 3] self.__offspring_size: 70
Current generation: 1
self.__offspring_size: 70
Current generation: 2
self.__offspring_size: 70
SOLUTION FOUND: [3 1 6 4 0 7 5 2] Current generation: 3
SOLUTION FOUND: [3 1 6 4 0 7 5 2] SOLUTION FOUND: [3 1 6 4 0 7 5 2] SOLUTION FOUND: [3 1 6 4 0 7 5 2] Time for new solution: 0.06144719800067833
SOLUTION FOUND: [3 1 6 4 0 7 5 2] SOLUTION FOUND: [3 1 6 4 0 7 5 2] self.__offspring_size: 70
Current generation: 4
self.__offspring_size: 70
SOLUTION FOUND: [3 1 6 2 5 7 4 0] Current generation: 5
SOLUTION FOUND: [3 1 6 2 5 7 4 0] SOLUTION FOUND: [3 1 6 2 5 7 4 0] SOLUTION FOUND: [3 1 6 2 5 7 4 0] Time for new solution: 0.08665592900069896
SOLUTION FOUND: [3 1 6 2 5 7 4 0] SOLUTION FOUND: [3 1 6 2 5 7 4 0] self.__offspring_size: 70
Current generation: 6
self.__o

In [33]:
evol_manager.show_solutions()

Solutions: 
solution [0]: [2 5 7 0 4 6 1 3].                     Generation: 0
solution [1]: [3 1 6 4 0 7 5 2].                     Generation: 3
solution [2]: [3 1 6 2 5 7 4 0].                     Generation: 5
solution [3]: [1 4 6 0 2 7 5 3].                     Generation: 8
solution [4]: [2 5 1 4 7 0 6 3].                     Generation: 12
solution [5]: [0 6 4 7 1 3 5 2].                     Generation: 21
solution [6]: [3 6 2 7 1 4 0 5].                     Generation: 23
solution [7]: [1 4 6 3 0 7 5 2].                     Generation: 51
solution [8]: [3 5 7 2 0 6 4 1].                     Generation: 51
solution [9]: [3 1 7 4 6 0 2 5].                     Generation: 65
